<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=8e8eb5800ad24277539f766a937117338be0bf2eaacbe291b6ee765a2507aabd
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.4
    Uninstalling protobuf-6.33.4:
      Successfully uninstalled protobuf-6.33.4


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-19 13:24:45
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.39 C
-------------------
Today PnL: -71.81 K (-0.51%)
Current PnL: -29.81 L (-19.01%)
CY Booked + Current PnL: -15.52 L (-9.9%)
-------------------
Total profit:  1.79 L
Total loss:  -31.59 L
-------------------
Total Booked + Current PnL: 11.46 L (8.83%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.26%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 95.16 L (68.33%)
Deployed:  1.30 C
Current:  1.39 C
CAGR/XIRR %: 3.76%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,-0.49,15.45,5.15,21.40,15781.0,40997.0,306423.0,7.09,59.0,X-LC,6.78,14.0,2.60,2.20,22.75,XY25,NTT,FMCG
17,COALINDIA,484.83,-0.19,5.79,12.70,19.23,20215.0,8714.0,159174.0,24.27,67.0,L-LC,10.45,182.0,0.43,1.14,28.30,XY25,ATH,MINING
77,TTKPRESTIG,770.00,-2.68,-24.48,32.53,0.09,24759.0,-24666.0,76111.0,76.73,29.0,M-SC,4.46,253.0,-1.00,0.55,0.00,OX40N,NTT,DURABLES
49,MASFIN,398.61,-0.03,-4.70,28.07,22.05,26210.0,-4605.0,93375.0,-17.80,45.0,H-SC,5.16,168.0,-0.18,0.67,35.73,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-5.92,-17.37,21.05,0.02,30362.0,-30322.0,144237.0,119.87,45.0,M-SC,9.32,240.0,-1.00,1.04,25.46,OX40N,NTT,PAINTS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,JSWINFRA,342.0,7.37,-5.80,23.60,16.43,49302.0,-12866.0,208908.0,-20.37,52.0,X-MC,7.50,66.0,-0.26,1.50,23.29,X40N,NTT,REALTY
84,WIPRO,420.0,2.79,9.70,57.04,72.27,106025.0,16430.0,185878.0,-6.08,57.0,M-LC,6.26,101.0,0.15,1.33,16.47,XR,NTT,IT
56,RAJOOENG,143.1,2.39,-35.12,115.19,39.61,76601.0,-36000.0,66500.0,-56.03,47.0,H-SC,30.20,136.0,-0.47,0.48,7.64,AR,ATH,MISC
33,HINDUNILVR,2922.0,2.03,-6.32,21.34,13.67,51391.0,-16243.0,240820.0,-13.28,59.0,X-LC,2.57,9.0,-0.32,1.73,13.69,XY25,NTT,FMCG
0,5PAISA,593.0,1.63,-27.50,55.72,12.90,88268.0,-60083.0,158413.0,133.17,61.0,H-SC,8.58,173.0,-0.68,1.14,31.72,OX40N,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-5.92,-17.37,21.05,0.02,30362.0,-30322.0,144237.0,119.87,45.0,M-SC,9.32,240.0,-1.00,1.04,25.46,OX40N,NTT,PAINTS
59,RELIANCE,1952.00,-3.62,-9.81,38.92,25.30,70546.0,-19705.0,181258.0,-12.63,25.0,X-LC,6.54,26.0,-0.28,1.30,21.41,XY25,BTT,REFINERIES
38,INDIAMART,4810.62,-3.57,-5.14,117.93,106.72,137969.0,-6344.0,116992.0,-53.11,50.0,H-SC,8.36,138.0,-0.05,0.84,17.03,AR,ATH,MISC
58,RELAXO,1176.00,-3.33,-51.39,213.31,52.31,150524.0,-74594.0,70566.0,-48.21,23.0,X-SC,8.48,92.0,-0.50,0.51,0.00,X40N,NTT,FOOTWEAR
28,GREENPANEL,537.00,-2.72,-43.14,151.75,43.14,157964.0,-78983.0,104095.0,133.20,29.0,M-SC,14.36,231.0,-0.50,0.75,0.00,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.0,-0.47,-0.04,20.79,20.74,42222.0,-82.0,203088.0,31.15,55.0,M-MC,5.13,189.0,-0.0,1.46,10.57,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-2.58,-3.77,116.06,107.91,164890.0,-5571.0,142073.0,-21.77,47.0,M-SC,9.97,220.0,-0.03,1.02,5.93,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-5.92,-17.37,21.05,0.02,30362.0,-30322.0,144237.0,119.87,45.0,M-SC,9.32,240.0,-1.00,1.04,25.46,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,-0.67,-19.75,64.53,32.03,68268.0,-26043.0,105792.0,-13.18,55.0,H-SC,6.41,157.0,-0.38,0.76,18.50,OX40N,NTT,IT
47,KANSAINER,340.00,-0.66,-23.27,47.89,13.47,99089.0,-62757.0,206910.0,-68.51,44.0,H-SC,4.10,159.0,-0.63,1.49,5.21,XY24,NTT,PAINTS
66,SIS,528.00,0.26,-24.00,59.56,21.26,50257.0,-26652.0,84380.0,1990.67,48.0,H-SC,4.76,163.0,-0.53,0.61,14.01,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,-0.19,5.79,12.70,19.23,20215.0,8714.0,159174.0,24.27,67.0,L-LC,10.45,182.0,0.43,1.14,28.30,XY25,ATH,MINING
84,WIPRO,420.00,2.79,9.70,57.04,72.27,106025.0,16430.0,185878.0,-6.08,57.0,M-LC,6.26,101.0,0.15,1.33,16.47,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,2.79,9.70,57.04,72.27,106025.0,16430.0,185878.0,-6.08,57.0,M-LC,6.26,101.0,0.15,1.33,16.47,XR,NTT,IT
25,FINCABLES,1641.55,-2.58,-3.77,116.06,107.91,164890.0,-5571.0,142073.0,-21.77,47.0,M-SC,9.97,220.0,-0.03,1.02,5.93,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-3.57,-5.14,117.93,106.72,137969.0,-6344.0,116992.0,-53.11,50.0,H-SC,8.36,138.0,-0.05,0.84,17.03,AR,ATH,MISC
37,IEX,219.00,-2.21,-7.56,60.78,48.63,112344.0,-15117.0,184837.0,-38.61,44.0,H-SC,13.44,137.0,-0.13,1.33,3.18,XR,NTT,MISC
49,MASFIN,398.61,-0.03,-4.70,28.07,22.05,26210.0,-4605.0,93375.0,-17.80,45.0,H-SC,5.16,168.0,-0.18,0.67,35.73,XR,ATH,FINANCE


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.0,-0.88,-31.76,130.34,57.19,290087.0,-103565.0,222562.0,-67.71,16.0,X-MC,11.76,58.0,-0.36,1.60,0.00,XY24,NTT,FMCG
43,ITC,452.0,0.94,-18.56,36.02,10.78,70500.0,-44593.0,195725.0,-51.37,17.0,X-LC,10.81,1.0,-0.63,1.41,0.94,X40,NTT,FMCG
11,BATAINDIA,2096.0,-1.22,-42.47,135.03,35.20,99948.0,-54651.0,74019.0,-2.96,22.0,X-SC,17.79,93.0,-0.55,0.53,0.00,X40,NTT,FOOTWEAR
58,RELAXO,1176.0,-3.33,-51.39,213.31,52.31,150524.0,-74594.0,70566.0,-48.21,23.0,X-SC,8.48,92.0,-0.50,0.51,0.00,X40N,NTT,FOOTWEAR
59,RELIANCE,1952.0,-3.62,-9.81,38.92,25.30,70546.0,-19705.0,181258.0,-12.63,25.0,X-LC,6.54,26.0,-0.28,1.30,21.41,XY25,BTT,REFINERIES


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4311.59,-1.51,-13.16,36.52,18.55,119955.0,-49774.0,328463.0,-25.24,49.0,X-LC,1.40,3.0,-0.41,2.36,11.75,X40,ATH,IT
45,JIOFIN,387.00,-1.40,-11.58,40.78,24.48,96410.0,-30951.0,236414.0,-13.03,30.0,X-LC,1.85,37.0,-0.32,1.70,37.03,XY24,BTT,FINANCE
78,UNITDSPR,1644.00,-1.16,-2.07,23.32,20.77,53471.0,-4851.0,229293.0,-10.63,36.0,X-MC,2.05,70.0,-0.09,1.65,4.70,X40N,NTT,BREWERIES
20,DABUR,735.00,-0.16,0.44,43.07,43.70,107317.0,1101.0,249169.0,-7.78,54.0,X-MC,2.12,73.0,0.01,1.79,15.56,XY24,BTT,FMCG
33,HINDUNILVR,2922.00,2.03,-6.32,21.34,13.67,51391.0,-16243.0,240820.0,-13.28,59.0,X-LC,2.57,9.0,-0.32,1.73,13.69,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-0.88,-31.76,130.34,57.19,290087.0,-103565.0,222562.0,-67.71,16.0,X-MC,11.76,58.0,-0.36,1.60,0.0,XY24,NTT,FMCG
8,BAJAJHFL,181.50,-0.97,-21.63,98.38,55.47,187576.0,-52621.0,190665.0,-30.61,28.0,X-MC,16.09,63.0,-0.28,1.37,0.0,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-1.22,-42.47,135.03,35.20,99948.0,-54651.0,74019.0,-2.96,22.0,X-SC,17.79,93.0,-0.55,0.53,0.0,X40,NTT,FOOTWEAR
53,PGHH,17907.65,-0.40,-8.60,46.82,34.19,91370.0,-18368.0,195152.0,-35.73,26.0,X-MC,4.88,60.0,-0.20,1.40,0.0,X40,ATH,FMCG
58,RELAXO,1176.00,-3.33,-51.39,213.31,52.31,150524.0,-74594.0,70566.0,-48.21,23.0,X-SC,8.48,92.0,-0.50,0.51,0.0,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-0.79,-31.15,107.47,42.84,48100.0,-20249.0,44757.0,-54.82,39.0,X-SC,23.79,83.0,-0.42,0.32,1.26,XY24,NTT,MISC
58,RELAXO,1176.00,-3.33,-51.39,213.31,52.31,150524.0,-74594.0,70566.0,-48.21,23.0,X-SC,8.48,92.0,-0.50,0.51,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.22,-42.47,135.03,35.20,99948.0,-54651.0,74019.0,-2.96,22.0,X-SC,17.79,93.0,-0.55,0.53,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-1.26,-5.66,31.53,24.08,37760.0,-7191.0,119759.0,-13.35,38.0,X-SC,6.00,91.0,-0.19,0.86,12.84,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.52,-17.01,72.71,43.32,98275.0,-27712.0,135160.0,-28.68,49.0,X-SC,8.11,90.0,-0.28,0.97,3.78,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,0.94,-18.56,36.02,10.78,70500.0,-44593.0,195725.0,-51.37,17.0,X-LC,10.81,1.0,-0.63,1.41,0.94,X40,NTT,FMCG
75,TMPV,600.00,-2.55,-29.37,74.11,22.98,118243.0,-66348.0,159550.0,-26.74,35.0,X-LC,3.41,2.0,-0.56,1.15,0.35,XY24,NTT,AUTO
73,TCS,4311.59,-1.51,-13.16,36.52,18.55,119955.0,-49774.0,328463.0,-25.24,49.0,X-LC,1.40,3.0,-0.41,2.36,11.75,X40,ATH,IT
81,VBL,671.64,-0.20,0.88,34.49,35.67,109891.0,2775.0,318617.0,-11.08,58.0,X-LC,2.98,5.0,0.03,2.29,14.85,X40N,ATH,FMCG
41,INFY,1972.00,-0.57,14.38,17.37,34.24,62164.0,44995.0,357883.0,-10.98,63.0,X-LC,7.48,6.0,0.72,2.57,23.85,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.48,-29.92,89.61,32.89,48488.0,-23096.0,54110.0,-768.50,72.0,L-MC,9.59,259.0,-0.48,0.39,49.01,XR,NTT,BANKS
13,BSOFT,831.70,-1.58,-22.72,95.12,50.79,103390.0,-31959.0,108694.0,-1.04,50.0,H-SC,9.80,171.0,-0.31,0.78,26.56,XR,ATH,IT
17,COALINDIA,484.83,-0.19,5.79,12.70,19.23,20215.0,8714.0,159174.0,24.27,67.0,L-LC,10.45,182.0,0.43,1.14,28.30,XY25,ATH,MINING
84,WIPRO,420.00,2.79,9.70,57.04,72.27,106025.0,16430.0,185878.0,-6.08,57.0,M-LC,6.26,101.0,0.15,1.33,16.47,XR,NTT,IT
36,ICICIPRULI,790.00,-2.60,9.28,19.62,30.73,42240.0,18285.0,215290.0,-16.22,49.0,X-MC,7.52,79.0,0.43,1.55,23.23,X40,ATH,INSURANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,1.63,-27.50,55.72,12.90,88268.0,-60083.0,158413.0,133.17,61.0,H-SC,8.58,173.0,-0.68,1.14,31.72,OX40N,NTT,FINANCE
40,INDUSINDBK,1800.00,-0.48,-29.92,89.61,32.89,48488.0,-23096.0,54110.0,-768.50,72.0,L-MC,9.59,259.0,-0.48,0.39,49.01,XR,NTT,BANKS
17,COALINDIA,484.83,-0.19,5.79,12.70,19.23,20215.0,8714.0,159174.0,24.27,67.0,L-LC,10.45,182.0,0.43,1.14,28.30,XY25,ATH,MINING
57,RECLTD,446.00,-0.47,-0.04,20.79,20.74,42222.0,-82.0,203088.0,31.15,55.0,M-MC,5.13,189.0,-0.00,1.46,10.57,XY25,NTT,FINANCE
62,SATIN,274.00,0.73,-15.83,80.38,51.83,186931.0,-43725.0,232559.0,-25.10,66.0,H-SC,3.86,148.0,-0.23,1.67,13.29,XY24,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.61
1,25,44.77
2,50,76.76


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.55
MC    30.64
LC    26.84
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.18
X40      24.16
X40N     12.87
XY25     10.86
XR        8.94
AR        8.39
OX40N     7.69
SR        0.94
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.25
X-LC    23.31
H-SC    22.80
M-SC    11.06
X-SC     7.90
H-MC     4.54
M-MC     1.46
M-LC     1.33
L-LC     1.14
H-LC     1.06
L-SC     0.79
L-MC     0.39
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.29,-9.38,45.26
IT,12.93,-18.73,78.04
FINANCE,11.12,-15.29,63.68
MISC,7.05,-32.47,86.66
ELECTRICAL,5.90,-14.36,56.24
PAINTS,5.78,-14.88,31.94
INSURANCE,4.69,-3.94,39.51
PHARMA,3.92,-7.64,41.53
BANKS,2.82,-36.09,118.55


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3325105.0,21
AR,1402788.0,10
XR,1247662.0,12
X40,1147740.0,15
X40N,951510.0,9
OX40N,723614.0,10
XY25,428415.0,7
SR,289474.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3621503.0,24
X-MC,1656028.0,16
M-SC,1517647.0,15
X-LC,970466.0,13
X-SC,819480.0,8
H-MC,446722.0,3
L-SC,181562.0,2
M-LC,106025.0,1
H-LC,85950.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1268341.0      6
           AR         943578.0      5
M-SC       XY24       867428.0      6
H-SC       XR         811830.0      7
X-MC       X40        544981.0      7
           XY24       463408.0      3
           X40N       419379.0      4
X-LC       X40        409002.0      6
M-SC       OX40N      323322.0      5
X-SC       X40N       323038.0      3
H-SC       OX40N      308280.0      4
X-SC       XY24       302685.0      3
H-SC       SR         289474.0      2
H-MC       AR         238132.0      2
X-MC       XY25       228260.0      2
X-LC       XY24       214653.0      2
           X40N       209093.0      2
H-MC       XY24       208590.0      1
X-SC       X40        193757.0      2
M-SC       XR         191769.0      2
X-LC       XY25       137718.0      3
M-SC       AR         135128.0      2
M-LC       XR         106025.0      1
L-SC       OX40N       92012.0      1
           XR          89550.0      1
H-LC       AR          85950.0      1
L-MC       XR          48488.0      1
M-MC       XY25        42222.0      1
L-LC       XY25        20215.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 38.0 seconds
